In [1]:
# Requires transformers>=4.51.0
# gpu visiableble setting
import os
import sys
def set_visible_gpus(gpu_ids):
    """
    Set the visible GPUs for the current process.
    
    Args:
        gpu_ids (str): Comma-separated string of GPU IDs to make visible.
    """
    if not isinstance(gpu_ids, str):
        raise ValueError("gpu_ids must be a string")
    
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_ids
    print(f"CUDA_VISIBLE_DEVICES set to: {gpu_ids}")

set_visible_gpus("1")
import torch
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]




tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-Embedding-0.6B', padding_side='left')
model = AutoModel.from_pretrained('Qwen/Qwen3-Embedding-0.6B')

# We recommend enabling flash_attention_2 for better acceleration and memory saving.
#model = AutoModel.from_pretrained('Qwen/Qwen3-Embedding-0.6B', attn_implementation="flash_attention_2", torch_dtype=torch.float16)
model = model.to("cuda")
max_length = 8192



CUDA_VISIBLE_DEVICES set to: 1


In [2]:
#load dataframe from 'mag7_aigc_results.parquet'
import pandas as pd
df = pd.read_parquet('mag7_aigc_results.parquet')


In [3]:
#scan df and find max length of column content  and idx
max_len = 0
max_idx = 0
for i in range(len(df)):
    if len(df.iloc[i, 0]) > max_len:
        max_len = len(df.iloc[i, 0])
        max_idx = i
print(f"Max length of content column: {max_len} at index {max_idx}")

Max length of content column: 39861 at index 4180


In [4]:
len(df.iloc[max_idx, 0])

39861

In [5]:
# import json, tqdm

# def get_embedding(texts, batch_size=32):
#     """
#     Generate embeddings for a list of texts using batch processing to avoid OOM.
    
#     Args:
#         texts (list): List of text strings to embed.
#         batch_size (int): Number of texts to process per batch.
    
#     Returns:
#         list: List of embedding vectors (as lists).
#     """
#     all_embeddings = []
    
#     # Process texts in batches
#     for i in tqdm.tqdm(range(0, len(texts), batch_size), desc="Processing batches"):
#         batch_texts = texts[i:i + batch_size]
        
#         # Tokenize the batch
#         encoded_input = tokenizer(
#             batch_texts,
#             padding=True,
#             truncation=True,
#             max_length=max_length,
#             return_tensors='pt'
#         ).to("cuda")
        
#         # Get model outputs
#         with torch.no_grad():
#             outputs = model(**encoded_input)
#             last_hidden_states = outputs.last_hidden_state
        
#         # Apply last token pooling
#         embeddings = last_token_pool(last_hidden_states, encoded_input['attention_mask'])
        
#         # Normalize embeddings
#         embeddings = F.normalize(embeddings, p=2, dim=1)
        
#         # Store embeddings
#         all_embeddings.extend(embeddings.cpu().numpy().tolist())
        
#         # Clear GPU memory
#         torch.cuda.empty_cache()
    
#     return all_embeddings

# def process_dataframe(df, batch_size=16):
#     """
#     Process the input DataFrame to generate embeddings for each column's text data.
    
#     Args:
#         df (pd.DataFrame): Input DataFrame with columns containing JSON strings.
#         batch_size (int): Number of texts to process per batch.
    
#     Returns:
#         pd.DataFrame: Output DataFrame with embeddings in the same structure.
#     """
#     output_df = pd.DataFrame(index=df.index, columns=df.columns)
    
#     for column in tqdm.tqdm(df.columns, desc="Processing columns"):
#         # Extract texts from the column
#         texts = []
#         for item in df[column]:
#             try:
#                 # Parse JSON string to dictionary and extract relevant text
                
#                 texts.append(item)
#             except json.JSONDecodeError:
#                 texts.append("")
        
#         # Generate embeddings for the texts
#         embeddings = get_embedding(texts, batch_size=batch_size)
        
#         # Store embeddings as JSON strings
#         output_df[column] = [json.dumps(emb) for emb in embeddings]
    
#     return output_df

In [6]:
import tqdm
import json, gc
def get_embedding(texts, batch_size=4):
    """
    Generate embeddings for a list of texts using batch processing to avoid OOM.
    
    Args:
        texts (list): List of text strings to embed.
        batch_size (int): Number of texts to process per batch.
    
    Returns:
        list: List of embedding vectors (as lists).
    """
    all_embeddings = []
    
    for i in tqdm.tqdm(range(0, len(texts), batch_size), desc="Processing batches"):
        batch_texts = texts[i:i + batch_size]
        
        # Tokenize the batch
        encoded_input = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors='pt'
        ).to("cuda")
        
        # Get model outputs
        with torch.no_grad():
            outputs = model(**encoded_input)
            last_hidden_states = outputs.last_hidden_state
        
        # Apply last token pooling
        embeddings = last_token_pool(last_hidden_states, encoded_input['attention_mask'])
        
        # Normalize embeddings
        embeddings = F.normalize(embeddings, p=2, dim=1)
        
        # Store embeddings
        all_embeddings.extend(embeddings.cpu().numpy().tolist())
        
        # Explicitly delete tensors and clear memory
        del encoded_input, outputs, last_hidden_states, embeddings
        torch.cuda.empty_cache()
        gc.collect()
        
        # Log memory usage for debugging
        #print(f"Batch {i//batch_size + 1}: GPU memory allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
    
    return all_embeddings

def process_dataframe(df, batch_size=16, chunk_size=8000):
    """
    Process the input DataFrame to generate embeddings for each column's text data.
    
    Args:
        df (pd.DataFrame): Input DataFrame with columns containing JSON strings.
        batch_size (int): Number of texts to process per batch.
        chunk_size (int): Number of rows to process per chunk.
    
    Returns:
        pd.DataFrame: Output DataFrame with embeddings in the same structure.
    """
    output_df = pd.DataFrame(index=df.index, columns=df.columns)
    
    for column in tqdm.tqdm(df.columns, desc="Processing columns"):
        # Process rows in chunks to reduce memory usage
        for start_idx in range(0, len(df), chunk_size):
            end_idx = min(start_idx + chunk_size, len(df))
            chunk_texts = []
            
            # Extract texts from the chunk
            for item in df[column].iloc[start_idx:end_idx]:
                try:
                    # Parse JSON string to dictionary and extract relevant text
                    #item length limited to 2048 characters
                    if isinstance(item, str):
                        item = item[:2048]  # Limit to 2048 characters
                    else:
                        item = str(item)[:2048]  # Convert to string and limit
                    chunk_texts.append(item)
                    
                except json.JSONDecodeError as e:
                    print(f"Warning: JSON decode error in column {column}: {e}")
                    chunk_texts.append("")
            
            # Generate embeddings for the chunk
            embeddings = get_embedding(chunk_texts, batch_size=batch_size)
            
            # Store embeddings in the output DataFrame
            output_df[column].iloc[start_idx:end_idx] = [json.dumps(emb) for emb in embeddings]
    
    return output_df

# Example usage
# input_df = pd.DataFrame({
#     'AAPL': ['```python\n{"股票代码": "AAPL", "日期": "2023-10-01"}\n```'] * 10000,
#     'MSFT': ['```python\n{"股票": "MSFT", "日期": "2023-10-01"}\n```'] * 10000,
# })
# output_df = process_dataframe(input_df, batch_size=4, chunk_size=1000)
# output_df.to_csv('embedded_data.csv', index=True)

In [7]:
embedding_df = process_dataframe(df)

Processing columns:   0%|          | 0/7 [00:00<?, ?it/s]

Processing batches: 100%|██████████| 310/310 [08:36<00:00,  1.67s/it]
/tmp/ipykernel_3111919/1863417875.py:91: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  output_df[column].iloc[start_idx:end_idx] = [json.dumps(emb) for emb in embeddings]


In [10]:
embedding_df.to_parquet('mag7_aigc_results_embedding.parquet', index=True)

In [17]:
embedding_df.head()

,AAPL,MSFT,GOOGL,AMZN,NVDA,META,TSLA
2006-03-27,"[-0.010473893024027348, 0.03431328013539314, -...","[-0.03791709616780281, 0.025187354534864426, -...","[-0.0244185458868742, 0.05697000026702881, -0....","[-0.036835409700870514, 0.09445228427648544, -...","[0.03612320497632027, 0.044321343302726746, -0...","[-0.03690069913864136, 0.016980359330773354, -...","[0.02404155023396015, 0.060831159353256226, -0..."
2006-03-28,"[0.004550086800009012, 0.05633486434817314, -0...","[-0.024092519655823708, 0.04828929901123047, -...","[-0.031112181022763252, 0.027739599347114563, ...","[-0.021372055634856224, 0.06737802922725677, -...","[-0.00650930218398571, 0.025904914364218712, -...","[-0.0016450012335553765, 0.03578471019864082, ...","[0.0465957410633564, 0.02111438475549221, -0.0..."
2006-03-29,"[-0.007588344160467386, 0.056205328553915024, ...","[-0.03325615078210831, 0.02914321795105934, -0...","[-0.015981199219822884, 0.024827459827065468, ...","[-0.026570795103907585, 0.07063396275043488, -...","[0.013584701344370842, 0.034102290868759155, -...","[-0.039836686104536057, 0.016013996675610542, ...","[0.03377041220664978, 0.06259383261203766, -0...."
2006-03-30,"[-0.024958888068795204, 0.06636907905340195, -...","[-0.042656801640987396, 0.010825795121490955, ...","[-0.038396649062633514, 0.03181783854961395, -...","[-0.01843183860182762, 0.04819761961698532, -0...","[0.03162645176053047, 0.03200918436050415, -0....","[-0.017583539709448814, 0.0014630352379754186,...","[0.048341259360313416, 0.016582613810896873, -..."
2006-03-31,"[-0.003448369214311242, 0.0480380542576313, -0...","[-0.026733791455626488, 0.024036569520831108, ...","[-0.032884642481803894, 0.03423741087317467, -...","[-0.04097476601600647, 0.04234614968299866, -0...","[0.006437849253416061, 0.03503504768013954, -0...","[-0.018855085596442223, 0.018116049468517303, ...","[0.014296467415988445, 0.06189926713705063, -0..."


In [22]:
import pandas as pd
import numpy as np
import ast

# Assuming A is your DataFrame with shape [100, 7], where each cell is an array of length 1024
# Example placeholder for A (replace with your actual DataFrame)
# A = pd.DataFrame(...)

# Reshape the DataFrame
def reshape_dataframe(df):
    # Get the number of rows and columns
    num_rows, num_cols = df.shape
    array_length = 1024  # Length of each array in a cell
    
    # Initialize an empty array to store the reshaped data
    reshaped_data = np.zeros((num_rows, num_cols * array_length))

    # Iterate over each row
    for i in range(num_rows):
        # Concatenate all arrays in the row
        # df.iloc[i, j]的内容是一个长度为1024的数组,使用字符串的形式，需要先转型成数组
        #df.iloc[i, j] = np.array(df.iloc[i, j])
        row_data = np.concatenate([ast.literal_eval(df.iloc[i, j]) for j in range(num_cols)])
        reshaped_data[i, :] = row_data
    
    # Create a new DataFrame with the reshaped data
    reshaped_df = pd.DataFrame(reshaped_data)
    reshaped_df.index = df.index  # Preserve the original index
    return reshaped_df

# Apply the reshaping function
reshaped_A = reshape_dataframe(embedding_df)

# Verify the shape
print(reshaped_A.shape)  # Should output (100, 7168) since 7 * 1024 = 7168

(4958, 7168)


In [26]:
reshaped_A.iloc[0,0]

-0.010473893024027348

In [27]:
import pandas as pd
import numpy as np

# Assuming DataFrame A is already loaded with shape [100, 7], where each cell is a numpy array of length 1024
# For demonstration, let's create a sample A if not provided
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA']

# Step 2: Create MultiIndex for columns
features = list(range(1024))
multi_index = pd.MultiIndex.from_product([tickers, features], names=['tickers', 'features'])

# Step 3: Create DataFrame B
reshaped_A.columns = multi_index

# Output the resulting DataFrame B


In [29]:
reshaped_A.to_parquet('mag7_aigc_results_embedding_reshaped.parquet', index=True)

In [15]:
import pandas as pd
import numpy as np

def reshape_dataframe(df):
    # 获取行数和列数
    num_rows, num_cols = df.shape
    array_length = 1024  # 每个单元格数组的预期长度
    
    # 初始化重塑后的数据数组
    reshaped_data = np.zeros((num_rows, num_cols * array_length))
    
    # 遍历每一行
    for i in range(num_rows):
        row_arrays = []
        for j in range(num_cols):
            # 提取单元格值
            cell = df.iloc[i, j]
            # 如果不是 NumPy 数组，尝试转换为 NumPy 数组
            if not isinstance(cell, np.ndarray):
                try:
                    cell = np.array(cell, dtype=float)
                except Exception as e:
                    raise ValueError(f"无法将 [{i}, {j}] 处的单元格转换为 NumPy 数组：{e}")
            
            # 检查数组是否为一维且长度正确
            if cell.ndim == 0:
                raise ValueError(f"[{i}, {j}] 处的单元格是零维数组（标量）。预期为一维长度为 {array_length} 的数组。")
            if cell.shape[0] != array_length:
                raise ValueError(f"[{i}, {j}] 处的单元格长度为 {cell.shape[0]}。预期长度为 {array_length}。")
            
            row_arrays.append(cell)
        
        # 拼接该行的所有数组
        try:
            row_data = np.concatenate(row_arrays)
        except ValueError as e:
            raise ValueError(f"第 {i} 行数组拼接失败：{e}")
        
        # 确保拼接后的行长度正确
        if row_data.shape[0] != num_cols * array_length:
            raise ValueError(f"第 {i} 行拼接后的数据长度为 {row_data.shape[0]}。预期为 {num_cols * array_length}。")
        
        reshaped_data[i, :] = row_data
    
    # 创建新的 DataFrame
    reshaped_df = pd.DataFrame(reshaped_data)
    
    return reshaped_df

# 应用重塑函数
try:
    reshaped_A = reshape_dataframe(embedding_df)
    print(reshaped_A.shape)  # 应输出 (100, 7168)，因为 7 * 1024 = 7168
except Exception as e:
    print(f"重塑过程中出错：{e}")

# 可选：根据股票代码分配列名
tickers = embedding_df.columns
new_columns = [f"{ticker}_{i}" for ticker in tickers for i in range(1024)]
reshaped_A.columns = new_columns

重塑过程中出错：无法将 [0, 0] 处的单元格转换为 NumPy 数组：could not convert string to float: '[-0.010473893024027348, 0.03431328013539314, -0.0013378848088905215, 0.023608606308698654, 0.062483858317136765, 0.009751654230058193, 0.02134564518928528, 0.025454487651586533, -0.05338792875409126, -0.049225810915231705, 0.04630224034190178, -0.028764022514224052, 0.033088065683841705, 0.002100762678310275, -0.03657372668385506, -0.00015793122292961925, -0.06244644150137901, -0.031432051211595535, -0.06327351182699203, 0.027225056663155556, 0.07172472774982452, -0.016771454364061356, -0.03620494157075882, 0.027216436341404915, -0.04180443286895752, -0.09769964963197708, 0.03399081528186798, 0.003334772540256381, -0.0521385483443737, 0.010352633893489838, 0.03474702686071396, -0.008120913058519363, -0.03469813987612724, -0.0065373810939490795, 0.00750644039362669, -0.0010695707751438022, 0.005285385996103287, 0.03388827294111252, -0.01708252914249897, 0.021881017833948135, -0.004998895805329084, 0.018596880137920

NameError: name 'reshaped_A' is not defined